In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import MDAnalysis as mda

/home/dbanerjee/.anaconda3/envs/md/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Process data using MDAnalysis

In [28]:
u_og =  mda.Universe("first.pdb","og.pdb")
ca_og = u_og.select_atoms("name CA")

assert ca_og.universe.trajectory.frame == 0

n_frames = ca_og.universe.trajectory.n_frames
n_atoms = ca_og.n_atoms
frame_start = 0
frame_end = n_frames

/home/dbanerjee/.anaconda3/envs/md/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


In [37]:
ca_og_pos = np.zeros((n_frames, 3*n_atoms))

for i,_ in enumerate(ca_og.universe.trajectory[frame_start:frame_end:1]):
    tmp = ca_og.positions.flatten()
    ca_og_pos[i,:] = tmp

(1000, 30)